In [ ]:
"""
Interacting with the infra-risk-vis autopackage API

This notebook will:
- Check the remote API is available
- Query the API for available boundaries
- Query the API for available datasets
- Post to the API to trigger an extract for a boundary and set of datasets
- Download files from the server to your local machine for further processing
"""

In [1]:
import os

import geopandas as gpd

import irv_autopkg_client

In [2]:
# declare some user inputs up front

# a country to request data for (ISO A3 code)
country_iso = "bgd"

# a location to store requested data
data_dir = "data"

In [3]:
# create a client object to establish a session
client = irv_autopkg_client.Client()

# is the API responding?
client.server_readiness()

True

In [5]:
# which boundaries can we create extracts for?
client.boundary_list()

[{'name': 'afg', 'name_long': 'Afghanistan'},
 {'name': 'ala', 'name_long': 'Aland'},
 {'name': 'alb', 'name_long': 'Albania'},
 {'name': 'dza', 'name_long': 'Algeria'},
 {'name': 'asm', 'name_long': 'American Samoa'},
 {'name': 'and', 'name_long': 'Andorra'},
 {'name': 'ago', 'name_long': 'Angola'},
 {'name': 'aia', 'name_long': 'Anguilla'},
 {'name': 'ata', 'name_long': 'Antarctica'},
 {'name': 'atg', 'name_long': 'Antigua and Barbuda'},
 {'name': 'arg', 'name_long': 'Argentina'},
 {'name': 'arm', 'name_long': 'Armenia'},
 {'name': 'abw', 'name_long': 'Aruba'},
 {'name': 'aus', 'name_long': 'Australia'},
 {'name': 'aut', 'name_long': 'Austria'},
 {'name': 'aze', 'name_long': 'Azerbaijan'},
 {'name': 'bhr', 'name_long': 'Bahrain'},
 {'name': 'bgd', 'name_long': 'Bangladesh'},
 {'name': 'brb', 'name_long': 'Barbados'},
 {'name': 'blr', 'name_long': 'Belarus'},
 {'name': 'bel', 'name_long': 'Belgium'},
 {'name': 'blz', 'name_long': 'Belize'},
 {'name': 'ben', 'name_long': 'Benin'},
 {'n

In [6]:
# which data processors can we deploy against some boundary?
client.dataset_list()

['gri_osm.roads_and_rail_version_1',
 'gridfinder.version_1',
 'isimp_drought.version_1',
 'jrc_ghsl_built_c.r2022_epoch2018_10m_mszfun',
 'jrc_ghsl_population.r2022_epoch2020_1km',
 'natural_earth_raster.version_1',
 'natural_earth_vector.version_1',
 'storm.global_mosaics_version_1',
 'test_fail_processor.version_1',
 'test_processor.version_1',
 'wri_aqueduct.version_2',
 'wri_powerplants.version_130']

In [7]:
# get information on a specific dataset
client.dataset("wri_aqueduct.version_2")

{'name': 'wri_aqueduct.version_2',
 'description': 'A Processor for WRI Aqueduct',
 'version': 'version_2',
 'status': '',
 'uri': '',
 'data_author': 'Ward, P.J., H.C. Winsemius, S. Kuzma, M.F.P. Bierkens, A. Bouwman, H. de Moel, A. Díaz Loaiza, et al.',
 'data_title': 'Aqueduct Flood Hazard Maps',
 'data_title_long': 'World Resource Institute - Aqueduct Flood Hazard Maps (Version 2, updated October 20, 2020)',
 'data_summary': 'World Resource Institute - Aqueduct Flood Hazard Maps (Version 2 (updated\nOctober 20, 2020)).  Inundation depth in meters for coastal and riverine\nfloods over 1km grid squares. 1 in 2 to 1 in 1000 year return periods.\nBaseline, RCP 4.5 & 8.5 emission scenarios. Current and future maps in 2030,\n2050 and 2080.',
 'data_citation': '\nWard, P.J., H.C. Winsemius, S. Kuzma, M.F.P. Bierkens, A. Bouwman, H. de Moel, A. Díaz Loaiza, et al. 2020.\nAqueduct Floods Methodology. Technical Note. Washington, D.C.: World Resources Institute. Available online at:\nwww.wri.

In [9]:
# submitting a job using one of those data processors
# as an example, all of the WRI Aqueduct flood maps for Egypt
job_id = client.job_submit(
    country_iso,
    [
        "gri_osm.roads_and_rail_version_1",
        "wri_aqueduct.version_2"
    ]
)

In [10]:
# get the status of the job
client.job_status(job_id)

{'job_group_status': 'COMPLETE',
 'job_group_percent_complete': 100,
 'job_group_processors': [{'job_id': 'a194fed9-c075-4c9d-9701-94f391aeb1af',
   'processor_name': 'gri_osm.roads_and_rail_version_1',
   'job_status': 'SKIPPED',
   'job_result': {'skipped': 'bgd.gri_osm.roads_and_rail_version_1 exists'}},
  {'job_id': 'a657c4d9-5bf9-445d-a63a-2e3ece47e29e',
   'processor_name': 'wri_aqueduct.version_2',
   'job_status': 'SKIPPED',
   'job_result': {'skipped': 'bgd.wri_aqueduct.version_2 exists'}}]}

In [11]:
# check if the job is complete
client.job_complete(job_id)

True

In [12]:
# get the boundary of the territory
boundary = client.boundary_geometry(country_iso)

# save it for later use
os.makedirs(os.path.join(data_dir, country_iso), exist_ok=True)
df = gpd.GeoDataFrame({"name": [country_iso], "geometry": [boundary]})
df.to_file(os.path.join(data_dir, country_iso, "territory.gpkg"))

In [27]:
# download some extracted data
client.extract_download(
    country_iso,
    data_dir,
    # there may be other datasets available, but only download the following
    dataset_filter=[
        "gri_osm.roads_and_rail_version_1",
        "wri_aqueduct.version_2"
    ],
    overwrite=True
)

2023-03-30 19:09:52,847 Downloading 1 of 380 to data/bgd/gri_osm_roads_and_rail_version_1/gri_osm-roads_and_rail_version_1-bgd.gpkg
2023-03-30 19:09:53,936 Downloading 2 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_MIROC-ESM-CHEM_2030_rp00005-bgd.tif
2023-03-30 19:09:54,058 Downloading 3 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_MIROC-ESM-CHEM_2080_rp00010-bgd.tif
2023-03-30 19:09:54,205 Downloading 4 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_00000NorESM1-M_2080_rp00005-bgd.tif
2023-03-30 19:09:54,328 Downloading 5 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_00000NorESM1-M_2050_rp00500-bgd.tif
2023-03-30 19:09:54,489 Downloading 6 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp00250-bgd.tif
2023-03-30 19:09:54,630 Downloading 7 of 380 to data/bgd/wri_aqueduct_version_2/wri_aque

2023-03-30 19:10:00,830 Downloading 55 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_0000HadGEM2-ES_2030_rp00025-bgd.tif
2023-03-30 19:10:00,947 Downloading 56 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_rcp4p5_wtsub_2050_rp0050_0-bgd.tif
2023-03-30 19:10:01,074 Downloading 57 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_rcp4p5_wtsub_2080_rp1000_0-bgd.tif
2023-03-30 19:10:01,204 Downloading 58 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_rcp8p5_wtsub_2050_rp0005_0-bgd.tif
2023-03-30 19:10:01,315 Downloading 59 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_00000NorESM1-M_2050_rp00025-bgd.tif
2023-03-30 19:10:01,435 Downloading 60 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00025-bgd.tif
2023-03-30 19:10:01,565 Downloading 61 of 380 to data/bgd/wri_aqueduct_version_2/w

2023-03-30 19:10:07,053 Downloading 108 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00250-bgd.tif
2023-03-30 19:10:07,182 Downloading 109 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_rcp8p5_wtsub_2080_rp0500_0-bgd.tif
2023-03-30 19:10:07,308 Downloading 110 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_historical_wtsub_2080_rp0002_0-bgd.tif
2023-03-30 19:10:07,408 Downloading 111 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_00IPSL-CM5A-LR_2030_rp00050-bgd.tif
2023-03-30 19:10:07,529 Downloading 112 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00100-bgd.tif
2023-03-30 19:10:07,646 Downloading 113 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_rcp4p5_wtsub_2030_rp0250_0-bgd.tif
2023-03-30 19:10:07,751 Downloading 114 of 380 to data/bgd/wri_aqueduct_

2023-03-30 19:10:13,148 Downloading 161 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_rcp4p5_wtsub_2050_rp0250_0-bgd.tif
2023-03-30 19:10:13,262 Downloading 162 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp00100-bgd.tif
2023-03-30 19:10:13,355 Downloading 163 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_0000GFDL-ESM2M_2030_rp00025-bgd.tif
2023-03-30 19:10:13,453 Downloading 164 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00250-bgd.tif
2023-03-30 19:10:13,566 Downloading 165 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_historical_000000000WATCH_1980_rp00100-bgd.tif
2023-03-30 19:10:13,667 Downloading 166 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_MIROC-ESM-CHEM_2030_rp01000-bgd.tif
2023-03-30 19:10:13,823 Downloading 167 of 380 to data/b

2023-03-30 19:10:19,636 Downloading 214 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00050-bgd.tif
2023-03-30 19:10:19,763 Downloading 215 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp00250-bgd.tif
2023-03-30 19:10:20,123 Downloading 216 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00050-bgd.tif
2023-03-30 19:10:20,254 Downloading 217 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_0000HadGEM2-ES_2080_rp00002-bgd.tif
2023-03-30 19:10:20,373 Downloading 218 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp00050-bgd.tif
2023-03-30 19:10:20,523 Downloading 219 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_MIROC-ESM-CHEM_2030_rp00100-bgd.tif
2023-03-30 19:10:20,679 Downloading 220 of 380 to da

2023-03-30 19:10:26,737 Downloading 267 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_rcp4p5_wtsub_2030_rp0002_0-bgd.tif
2023-03-30 19:10:26,942 Downloading 268 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_00000NorESM1-M_2050_rp00050-bgd.tif
2023-03-30 19:10:27,096 Downloading 269 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_0000GFDL-ESM2M_2030_rp00050-bgd.tif
2023-03-30 19:10:27,229 Downloading 270 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00100-bgd.tif
2023-03-30 19:10:27,370 Downloading 271 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_MIROC-ESM-CHEM_2030_rp00050-bgd.tif
2023-03-30 19:10:32,538 Downloading 272 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_0000HadGEM2-ES_2030_rp00010-bgd.tif
2023-03-30 19:10:32,664 Downloading 273 of 380 to data/bgd/w

2023-03-30 19:10:44,121 Downloading 320 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00010-bgd.tif
2023-03-30 19:10:44,246 Downloading 321 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_MIROC-ESM-CHEM_2030_rp01000-bgd.tif
2023-03-30 19:10:44,361 Downloading 322 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_historical_wtsub_2050_rp0100_0-bgd.tif
2023-03-30 19:10:44,506 Downloading 323 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00500-bgd.tif
2023-03-30 19:10:44,626 Downloading 324 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_0000GFDL-ESM2M_2030_rp00025-bgd.tif
2023-03-30 19:10:44,765 Downloading 325 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00025-bgd.tif
2023-03-30 19:10:44,898 Downloading 326 of 380 to data/b

2023-03-30 19:10:50,903 Downloading 373 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_00000NorESM1-M_2030_rp00010-bgd.tif
2023-03-30 19:10:51,016 Downloading 374 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_0000GFDL-ESM2M_2080_rp00025-bgd.tif
2023-03-30 19:10:51,135 Downloading 375 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_MIROC-ESM-CHEM_2080_rp00050-bgd.tif
2023-03-30 19:10:51,260 Downloading 376 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00010-bgd.tif
2023-03-30 19:10:51,367 Downloading 377 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inuncoast_rcp8p5_wtsub_2050_rp0010_0-bgd.tif
2023-03-30 19:10:51,472 Downloading 378 of 380 to data/bgd/wri_aqueduct_version_2/wri_aqueduct-version_2-inunriver_rcp4p5_0000GFDL-ESM2M_2030_rp00005-bgd.tif
2023-03-30 19:10:51,588 Downloading 379 of 380 to data/bgd/w